# Rule Learner algorithm (PRISM): Notes

Use lecture slides, book chapters or the original paper to understand the PRISM algorithm. 

Read this notebook. Then, __IN A SEPARATE NOTEBOOK__, implement the algorithm. Implement it step-by-step, add explanations of each step, keep the code clean and modular. Finally, test the correctness of your algorithm on the contact lenses [dataset](contact_lenses.csv) discussed in class, to see if it produces the same rules.


## 1. Creating some classes

These are the classes you made for the lab, thank you!

In [7]:
class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class
        self.accuracy = 0
        self.coverage = 0

    def addCondition(self, condition):
        self.conditions.append(condition)

    def setParams(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage
    
    # Human-readable printing of this Rule
    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)

The list of conditions contains several objects of class _Condition_. 

Each condition includes the _attribute name_ and the _value_. 

If the _value_ is numeric, then the condition also includes an additional field `true_false` which means the following: 
- *if true_false == True then values are >= value* 
- *if true_false == False then values are < value*
- If *true_false is None*, then this condition is simply of form *categorical attribute = value*.

In [8]:
class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            return "{}>={}:{}".format(self.attribute, self.value, self.true_false)

## 2. The Algorithm

I'll be using the approach you laid out in the Algorithm Notes section. There are some differences, notably in the stucture. I noticed that refining rules and finding the first rule are very similar, so I made them one function (refine_rule()). 

There is maybe one big decision I made that could differ from the normal algorithm. I added a condition to break refining if we ever fail to find an additional rule that improves our accuracy. So if a rule starts out at 70% accruate, and any other possible rule we could add it to keeps the overall rules accuracy at 70%, we do not include it. This is not quite what the algorithm does, but it will make the examples work better. The main motivation for my adding it, is if we do not include this and min_coverage is 1 (like it is in the eye example), then the PRISM algorithm will often give complicated rules singling out a single person.  

For the larger datasets in the next section, I will repeat all calculations with both an algorithm including the refinement stopper and one without. 



In [7]:
def refine_rule(columns, covered_subset, class_label, min_coverage=30, min_accuracy=0.6, current_rule = None, colTypes = None):
    
    # If we're refining from None, create an empty Rule
    if (current_rule == None):
        current_rule = Rule(class_label)
        

    # Some helpful variables for tracking 
    bestAcc = 0
    bestCov = 0
    bestCondition = None
    curAcc = 0
    curCov = 0 

    # We start by testing all possible columns/traits until we find the best one:
    for trait in columns:
        if trait == columns[-1]:
            continue
    
        # If the trait is numeric
        if colTypes[trait].loc['type'] == "numeric":
            # We'll split by less than and ge. 
            
            # Less than:
            
            for level in covered_subset[trait].unique().tolist():
                ruleSubset = covered_subset[covered_subset[trait] < level] # Split by less than
                
                # Make sure we don't divide by 0!
                if (len(ruleSubset) > 0):
                    curAcc = len(ruleSubset[ruleSubset[columns[-1]] == class_label]) / len(ruleSubset)
                    curCov = len(ruleSubset[ruleSubset[columns[-1]] == class_label])
                    # Evaluate effectiveness of the rule
                    if (curCov >= min_coverage):
                        if (curAcc > bestAcc):
                            bestAcc = curAcc
                            bestCov = curCov
                            bestCondition = Condition(trait, level, False)
                        if (curAcc == bestAcc):
                            if (curCov > bestCov):
                                bestCov = curCov
                                bestCondition = Condition(trait, level, False)
                
                # Now check the >= case:
                ruleSubset = covered_subset[covered_subset[trait] >= level] # Split by ge
                if (len(ruleSubset) > 0):
                    curAcc = len(ruleSubset[ruleSubset[columns[-1]] == class_label]) / len(ruleSubset)
                    curCov = len(ruleSubset[ruleSubset[columns[-1]] == class_label])
                    # Evaluate goodness of the rule
                    if (curCov >= min_coverage):
                        if (curAcc > bestAcc):
                            bestAcc = curAcc
                            bestCov = curCov
                            bestCondition = Condition(trait, level, True)
                        if (curAcc == bestAcc):
                            if (curCov > bestCov):
                                bestCov = curCov
                                bestCondition = Condition(trait, level, True)
                            
            continue # Go to the next loop
        
         
        # If our trait is categorical:
        for typeTrait in covered_subset[trait].unique().tolist():
            ruleSubset = covered_subset[covered_subset[trait] == typeTrait]
            
            # Now test for accuracy and coverage of the rule:
            curAcc = len(ruleSubset[ruleSubset[columns[-1]] == class_label]) / len(ruleSubset)
            curCov = len(ruleSubset[ruleSubset[columns[-1]] == class_label])
            if (curCov >= min_coverage):
                if (curAcc > bestAcc):
                    bestAcc = curAcc
                    bestCov = curCov
                    bestCondition = Condition(trait, typeTrait)

                # If we have another rule with the same accuracy, 
                # choose the one with the better coverage
                if (curAcc == bestAcc):
                    if curCov > bestCov:
                        bestCov = curCov
                        bestCondition = Condition(trait, typeTrait)

    # If refining does not improve our accuracy, we break
    if (bestAcc == current_rule.accuracy):
        return(None, None)
    
    # If we have no rule, break
    if (bestCondition == None):
        return(None, None)
    
    # Otherwise, add the rule to the list and
    # shorten the subset
    current_rule.addCondition(bestCondition)
    current_rule.setParams(bestAcc, bestCov)
    
    # Numeric makes things a little stranger
    if colTypes[bestCondition.attribute].loc['type'] == "numeric":
        if bestCondition.true_false:
            covered_subset = covered_subset[covered_subset[bestCondition.attribute] >= bestCondition.value]
        else:
            covered_subset = covered_subset[covered_subset[bestCondition.attribute] < bestCondition.value]
    else:
        covered_subset = covered_subset[covered_subset[bestCondition.attribute] == bestCondition.value]
    return (current_rule, covered_subset)
                            
    
    

# Ok, now we can learn_one_rule!

def learn_one_rule(columns, data, class_label, min_coverage=30, min_accuracy=0.6, colType = None):
    # Make a copy of columns so we can delete from it. 
    # Otherwise we'd be destorying columns
    columnsCopy = columns.copy().tolist()
    
    # Refine once from None, this will create the best first rule we want
    current_rule, covered_subset = refine_rule(columnsCopy, data.copy(), class_label, min_coverage, min_accuracy, None, colType)
    
    # If we got None out, it means there is no rule that meets min_coverage
    if (current_rule == None):
        return (None, None)
    
    # Remove the trait from the list of eligbile traits
    columnsCopy.remove(current_rule.conditions[-1].attribute)
    
    # We will refine while our rule isn't perfect and there are 
    # traits left to add
    while current_rule.accuracy < 1 and len(columnsCopy) > 1:
        temp_rule, temp_subset = refine_rule(columnsCopy, covered_subset,
                                             class_label, min_coverage, min_accuracy, current_rule, colType)
        # Just remove the trait we just used
        if (temp_rule != None):
            current_rule = temp_rule
            covered_subset = temp_subset
            columnsCopy.remove(current_rule.conditions[-1].attribute)
        else:
            break


    if current_rule.accuracy < min_accuracy:
        return (None, None)
    
    return (current_rule, covered_subset)

Finally, the main algorithm `learn_rules` takes as parameters list of columns, with the last column representing the class label, and the original data in form of pandas dataframe. Optionally, you should be able to specify the list of classes that you are interested to explore first. Two optional threshold parameters `min_coverage` and `min_accuracy` set up the conditions of rule's validity for a given dataset.

We had to implement the step where we make the data domain smaller, I just dropped by matching up the ID's of the two datasets. 

In [12]:
import pandas as pd
import numpy as np

    
    # We can pre-process to determine if something is a numeric trait. 
    # We'll use two rules here:
    # 1. If the Trait is all numeric
    # 2. If there are a certain number of unique oberservations
    #    in the dataset, say more than 10. 
    # 
    # Returns a Pandas Dataframe with the columns of `columns`
    # and a row of "type" with the proper type of the variable
def learn_categories(columns, data, min_threshold = 10):
    types = []
    cols = data.columns
    for column in cols:
        if data[column].dtype == 'int64' and len(data[column].unique()) >= min_threshold::
            types.append("numeric")
        else:
            types.append("categorical")

    colType = pd.DataFrame(columns = cols)

    colType.loc['type'] = types
    return colType
    
    
def learn_rules (columns, data, classes=None, 
    min_coverage = 30, min_accuracy = 0.6):
    # List of final rules
    rules = []
    
    # If list of classes of interest is not provided - it is extracted from the last column of data
    if classes is not None:
        class_labels = classes
    else:
        class_labels = data[columns[-1]].unique().tolist()

    current_data = data.copy()

    # Get the Column types (categorical vs. numeric)
    colType = learn_categories(columns, data)

    
    
    # This follows the logic of the original PRISM algorithm
    # It processes each class in turn. 
    for class_label in class_labels:
        done = False
        while len(current_data) > min_coverage and not done:
            # Learn one rule 
            rule, subset = learn_one_rule(columns, current_data, class_label, min_coverage, min_accuracy, colType)
            
            # If the best rule does not pass the coverage threshold - we are done with this class
            if rule is None:
                break

            # If we get the rule with accuracy and coverage above threshold
            
            if rule.accuracy >= min_accuracy:
                rules.append(rule)

                # We're going to say that `subset` is the rows that need to be removed. 
                # We can drop by matching up the indices
                current_data = current_data.drop(index=subset.index)
                   
            else:
                done = True         
                
    return rules

## 3. Correctness test  

Made no changes to the code here!

Test your algorithm on the original dataset from the PRISM paper.

The dataset was downloaded from [here](https://archive.ics.uci.edu/ml/datasets/Lenses). The CSV version is included in this repository.

**Attribute Information**:

3 Classes:
- __1__ : the patient should be fitted with __hard__ contact lenses,
- __2__ : the patient should be fitted with __soft__ contact lenses,
- __3__ : the patient should __not__ be fitted with contact lenses.


Attributes:
1. age of the patient: (1) young, (2) pre-presbyopic, (3) presbyopic
2. spectacle prescription:  (1) myope, (2) hypermetrope
3. astigmatic:     (1) no, (2) yes
4. tear production rate:  (1) reduced, (2) normal

Presbyopia is physiological insufficiency of accommodation associated with the aging of the eye that results in progressively worsening ability to focus clearly on close objects. So "age=presbiopic" means old.

Hypermetropia: far-sightedness, also known as long-sightedness - cannot see close.
Myopia: nearsightedness - cannot see at distance.

In [13]:
import pandas as pd
import numpy as np

data_file = "contact_lenses.csv"
data = pd.read_csv(data_file, index_col=['id'])
data.columns


Index(['age', 'spectacles', 'astigmatism', 'tear production rate',
       'lenses type'],
      dtype='object')

We can replace numbers with actual values - for clarity.

In [14]:
# classes
conditions = [ data['lenses type'].eq(1), data['lenses type'].eq(2), data['lenses type'].eq(3)]
choices = ["hard","soft","none"]

data['lenses type'] = np.select(conditions, choices)

# age groups
conditions = [ data['age'].eq(1), data['age'].eq(2), data['age'].eq(3)]
choices = ["young","medium","old"]

data['age'] = np.select(conditions, choices)

# spectacles
conditions = [ data['spectacles'].eq(1), data['spectacles'].eq(2)]
choices = ["nearsighted","farsighted"]

data['spectacles'] = np.select(conditions, choices)

# astigmatism
conditions = [ data['astigmatism'].eq(1), data['astigmatism'].eq(2)]
choices = ["no","yes"]

data['astigmatism'] = np.select(conditions, choices)

# tear production rate
conditions = [ data['tear production rate'].eq(1), data['tear production rate'].eq(2)]
choices = ["reduced","normal"]

data['tear production rate'] = np.select(conditions, choices)

The test (do not run it before you finished the implementation of the rule learning algorithm):

In [15]:
column_list = data.columns.to_numpy()
rules = learn_rules (column_list, data, None, 1, 0.95)
for rule in rules[:20]:
    print(rule)

age
spectacles
astigmatism
tear production rate
lenses type
If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0
If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0
If [astigmatism=no, age=young] then soft. Coverage:1, accuracy: 1.0
If [astigmatism=no, age=medium] then soft. Coverage:1, accuracy: 1.0
If [age=young] then hard. Coverage:2, accuracy: 1.0
If [spectacles=nearsighted, astigmatism=yes] then hard. Coverage:2, accuracy: 1.0


## My Results

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0

If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0

If [astigmatism=no, age=young] then soft. Coverage:1, accuracy: 1.0

If [astigmatism=no, age=medium] then soft. Coverage:1, accuracy: 1.0

If [age=young] then hard. Coverage:2, accuracy: 1.0

If [spectacles=nearsighted, astigmatism=yes] then hard. Coverage:2, accuracy: 1.0

They match!!

## Your Results
My results are given below for comparison:

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0

If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0

If [astigmatism=no, age=young] then soft. Coverage:1, accuracy: 1.0

If [astigmatism=no, age=medium] then soft. Coverage:1, accuracy: 1.0

If [age=young] then hard. Coverage:2, accuracy: 1.0

If [spectacles=nearsighted, astigmatism=yes] then hard. Coverage:2, accuracy: 1.0



## Repeating the Above Step removing the Refining Stop Condition

As I mentioned, the refining stop condition makes things trickier for the glasses example specifically, so I'll illustrate what I found here. 

In [9]:
def refine_rule_rf(columns, covered_subset, class_label, min_coverage=30, min_accuracy=0.6, current_rule = None):
    
    if (current_rule == None):
        current_rule = Rule(class_label)
        

    bestAcc = 0
    bestCov = 0
    bestCondition = None
    curAcc = 0
    curCov = 0 

    # We start by testing all possible columns/traits until we find the best one:
    for trait in columns:
        if trait == columns[-1]:
            continue
        for typeTrait in covered_subset[trait].unique().tolist():
            ruleSubset = covered_subset[covered_subset[trait] == typeTrait]
            # Now test for accuracy of the rule:
            # We can use .size which normally 
            # returns total number of element in the dataframe
            # but since we would divide both numbers by the number of columns
            # to get accuracy, we can just do this

            curAcc = len(ruleSubset[ruleSubset[columns[-1]] == class_label]) / len(ruleSubset)
            curCov = len(ruleSubset[ruleSubset[columns[-1]] == class_label])
            if (curCov >= min_coverage):
                if (curAcc > bestAcc):
                    bestAcc = curAcc
                    bestCov = curCov
                    bestCondition = Condition(trait, typeTrait)

                # If we have another rule with the same accuracy, 
                # choose the one with the better coverage
                if (curAcc == bestAcc):
                    if curCov > bestCov:
                        bestCov = curCov
                        bestCondition = Condition(trait, typeTrait)


    # If we have no rule, break
    if (bestCondition == None):
        return(None, None)
    # Otherwise, add the rule to the list and
    # shorten the subset
    current_rule.addCondition(bestCondition)
    current_rule.setParams(bestAcc, bestCov)
    covered_subset = covered_subset[covered_subset[bestCondition.attribute] == bestCondition.value]
    return (current_rule, covered_subset)
                            
    
    

# Ok, now we can learn_one_rule!

def learn_one_rule_rf(columns, data, class_label, min_coverage=30, min_accuracy=0.6):
    # Make a copy of columns so we can delete from it. 
    # Otherwise we'd be destorying columns
    columnsCopy = columns.copy()
    
    # Refine once, this will create the rule we want
    current_rule, covered_subset = refine_rule_rf(columnsCopy, data.copy(), class_label, min_coverage, min_accuracy, None)
    
    # If we got None out, it means there is no rule that meets min_coverage
    if (current_rule == None):
        return (None, None)
    
    # Remove the trait from the list of eligbile traits
    columnsCopy.remove(current_rule.conditions[-1].attribute)
    
    # We will refine while our rule isn't perfect and there are 
    # traits left to add
    while current_rule.accuracy < 1 and len(columnsCopy) > 1:
        temp_rule, temp_subset = refine_rule_rf(columnsCopy, covered_subset,
                                             class_label, min_coverage, min_accuracy, current_rule)
        if (temp_rule != None):
            current_rule = temp_rule
            covered_subset = temp_subset
            columnsCopy.remove(current_rule.conditions[-1].attribute)
        else:
            break


    if current_rule.accuracy < min_accuracy:
        return (None, None)
    
    return (current_rule, covered_subset)

def learn_rules_rf (columns, data, classes=None, 
                 min_coverage = 30, min_accuracy = 0.6):
    # List of final rules
    rules = []
    
    # If list of classes of interest is not provided - it is extracted from the last column of data
    if classes is not None:
        class_labels = classes
    else:
        class_labels = data[columns[-1]].unique().tolist()

    current_data = data.copy()
    
    # This follows the logic of the original PRISM algorithm
    # It processes each class in turn. 
    for class_label in class_labels:
        done = False
        while len(current_data) > min_coverage and not done:
            # Learn one rule 
            rule, subset = learn_one_rule_rf(columns, current_data, class_label, min_coverage, min_accuracy)
            
            # If the best rule does not pass the coverage threshold - we are done with this class
            if rule is None:
                break

            # If we get the rule with accuracy and coverage above threshold
            
            if rule.accuracy >= min_accuracy:
                rules.append(rule)

                # We're going to say that `subset` is the rows that need to be removed. 
                # We can drop by matching up the indices
                current_data = current_data.drop(index=subset.index)
                   
            else:
                done = True         
                
    return rules

In [11]:
column_list = data.columns.to_numpy().tolist()
rules = learn_rules_rf (column_list, data, None, 1, 0.95)
for rule in rules[:20]:
    print(rule)

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0
If [age=old, tear production rate=normal, spectacles=nearsighted, astigmatism=no] then none. Coverage:1, accuracy: 1.0
If [spectacles=farsighted, astigmatism=yes, age=medium] then none. Coverage:1, accuracy: 1.0
If [age=old, spectacles=farsighted, astigmatism=yes] then none. Coverage:1, accuracy: 1.0
If [astigmatism=no] then soft. Coverage:5, accuracy: 1.0
If [astigmatism=yes] then hard. Coverage:4, accuracy: 1.0


## Discussion

So we found a few differences, notably our dataset now rules covering all 24 points in the dataset, so we're perfectly identified here. The first rule is the same, but the second rule now goes through a bunch of different traits. We also have more complicated rules than we did before. 

Copyright &copy; 2020 Marina Barsky. All rights reserved.